# Chapter 15: Distributing TensorFlow Across Devices and Servers

Notebook ini merupakan hasil reproduksi dan penjelasan teori dari **Bab 15 - Distributing TensorFlow Across Devices and Servers** dari buku *Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow (2nd Edition)* oleh Aurélien Géron.

📌 Bab ini membahas cara melakukan distribusi training model TensorFlow menggunakan beberapa perangkat (GPU/TPU) atau bahkan server yang berbeda.

---


## Ringkasan Teori Bab 15: Distributing TensorFlow Across Devices and Servers

### 1. Mengapa Perlu Distribusi?

- Model deep learning skala besar membutuhkan waktu lama untuk dilatih
- Dataset besar sulit ditangani oleh satu GPU
- Distribusi membantu mempercepat training dan mengurangi bottleneck

---

### 2. Deteksi Perangkat

```python
tf.config.list_physical_devices()
```

TensorFlow secara otomatis dapat menggunakan CPU, GPU, atau TPU jika tersedia.

---

### 3. Strategi Distribusi

TensorFlow menyediakan beberapa strategi:

- **`MirroredStrategy`**: Salin model ke beberapa GPU di satu mesin
- **`MultiWorkerMirroredStrategy`**: Distribusi antar beberapa worker/machine
- **`TPUStrategy`**: Untuk menjalankan di TPU (Google Cloud, Colab)
- **`CentralStorageStrategy`**: Parameter disimpan di host dan dikirim ke worker saat training

---

### 4. Penggunaan `MirroredStrategy`

```python
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = keras.models.Sequential([...])
    model.compile(...)
```

- Parameter model disalin ke semua GPU
- Gradien digabungkan dan dibagi otomatis
- Kompatibel dengan `.fit()`

---

### 5. Dataset Sharding Otomatis

Saat menggunakan strategi distribusi, dataset secara otomatis akan di-*shard* (dibagi) ke tiap worker agar efisien dan sinkron.

```python
dataset = dataset.batch(32)
```

---

### 6. MultiWorkerMirroredStrategy

Gunakan `TF_CONFIG` environment variable untuk mendeskripsikan cluster (beberapa mesin):
```json
{
  "cluster": {
    "worker": ["host1:port", "host2:port"]
  },
  "task": {"type": "worker", "index": 0}
}
```

- Butuh komunikasi antar node
- Cocok untuk cloud/enterprise training

---

### 7. Tips Distribusi:

- Jangan gunakan `.fit()` tanpa `strategy.scope()`!
- Simpan model di dalam `scope`
- Gunakan `strategy.run(fn)` jika menulis loop manual
- Gunakan GPU yang identik untuk sinkronisasi optimal

---

### 8. TPU Strategy (Opsional)

```python
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
strategy = tf.distribute.TPUStrategy(resolver)
```

- Biasanya hanya tersedia di Google Colab Pro atau GCP TPU

---

📌 Ringkasan:
Distribusi model memungkinkan training skala besar secara efisien, baik di laptop multi-GPU, server, atau cloud TPU/GPU farms.

---


## Implementasi Distribusi dengan MirroredStrategy

Strategi ini secara otomatis membagi data dan mereplikasi model ke semua GPU dalam satu mesin. Cocok digunakan di Colab Pro atau PC dengan banyak GPU.

---


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Dummy dataset
X = np.random.rand(1000, 10)
y = (X[:, 0] + X[:, 1] * 2 + np.random.rand(1000)) > 1.5
y = y.astype(np.float32)

# Gunakan strategi distribusi
strategy = tf.distribute.MirroredStrategy()
print(f"Jumlah perangkat: {strategy.num_replicas_in_sync}")

# Buat dan latih model di dalam scope
with strategy.scope():
    model = keras.models.Sequential([
        keras.layers.Dense(32, activation='relu', input_shape=(10,)),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Latih model
model.fit(X, y, epochs=5, batch_size=64)

### Penjelasan:
- `MirroredStrategy()` secara otomatis mencari semua GPU yang tersedia
- `strategy.scope()` wajib digunakan saat membuat dan melatih model
- Model akan didistribusikan secara sinkron (gradien digabung dan dibagi rata)
- Training bisa dipercepat 2x, 4x, atau lebih tergantung jumlah GPU

📌 Gunakan GPU identik agar sinkronisasi tidak menjadi bottleneck.

---
